In [39]:
import sys
import os
script_dir = os.getcwd()
sys.path.append(script_dir)

In [40]:
from datasets import load_dataset

In [45]:
from datasets import load_dataset

# Carregar a base de dados
dataset = load_dataset("dair-ai/emotion")


ValueError: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion.
Please pass the argument `trust_remote_code=True` to allow custom code to be run.

In [42]:
from sentence_transformers import InputExample

# Preparar exemplos de treino
train_examples = []
for i in range(len(dataset['train'])):
    text = dataset['train'][i]['text']
    label = dataset['train'][i]['label']
    train_examples.append(InputExample(texts=[text], label=label))

# Preparar exemplos de validação
val_examples = []
for i in range(len(dataset['validation'])):
    text = dataset['validation'][i]['text']
    label = dataset['validation'][i]['label']
    val_examples.append(InputExample(texts=[text], label=label))


In [43]:
from sentence_transformers import SentenceTransformer, losses
from torch.utils.data import DataLoader

# Carregar o modelo pré-treinado
model = SentenceTransformer('distilbert-base-uncased')

# Criar DataLoaders para treino e validação
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
val_dataloader = DataLoader(val_examples, shuffle=False, batch_size=16)

# Definir a função de perda
train_loss = losses.CosineSimilarityLoss(model=model)


2024-08-01 01:36:56,746 - Use pytorch device_name: cpu
2024-08-01 01:36:56,748 - Load pretrained SentenceTransformer: distilbert-base-uncased
2024-08-01 01:36:57,257 - No sentence-transformers model found with name distilbert-base-uncased. Creating a new one with mean pooling.


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

c:\Users\Pichau\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Pichau\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [44]:
# Treinar o modelo
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=4,
          warmup_steps=100,
          evaluator=None,  # Você pode adicionar um evaluator se quiser monitorar a performance no conjunto de validação
          output_path='output/emotion-model')


  0%|          | 0/4000 [00:00<?, ?it/s]

IndexError: list index out of range

In [ ]:
# Carregar o modelo fine-tuned
finetuned_model = SentenceTransformer('output/emotion-model')

# Avaliar o desempenho
from sklearn.metrics import accuracy_score

val_texts = [example.texts[0] for example in val_examples]
val_labels = [example.label for example in val_examples]
val_predictions = finetuned_model.predict(val_texts)

# Calcular a acurácia
accuracy = accuracy_score(val_labels, val_predictions)
print("Acurácia na validação:", accuracy)
